In [2]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from nltk.stem.porter import PorterStemmer
import torch, nltk, re, json, os, random


stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0

    return bag

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
class NeuralNet(nn.Module):
	def __init__(self, input_size, hidden_size, num_classes):
		super(NeuralNet, self).__init__()
		self.l1 = nn.Linear(input_size, hidden_size)
		self.l2 = nn.Linear(hidden_size, hidden_size)
		self.l3 = nn.Linear(hidden_size, num_classes)
		self.relu = nn.ReLU()

	def forward(self, x):
		out = self.l1(x)
		out = self.relu(out)
		out = self.l2(out)
		out = self.relu(out)
		out = self.l3(out)

		#no activation no softmax

		return out

In [5]:
def train_model():
	with open('intents.json') as f:
		intents = json.load(f)

	all_words = []
	tags = []
	xy = []

	for intent in intents['intents']:
		tag = intent['tag']
		tags.append(tag)
		for pattern in intent['patterns']:
			w = tokenize(pattern)
			all_words.extend(w)
			xy.append((w, tag))

	ignore_words = ['?', '!', ',', '.']
	all_words = [stem(w) for w in all_words if w not in ignore_words]
	all_words = sorted(set(all_words))
	tags = sorted(set(tags))

	x_train = []
	y_train = []

	for (pattern_sentence, tag) in xy:
		bag = bag_of_words(pattern_sentence, all_words)
		x_train.append(bag)

		label = tags.index(tag)
		y_train.append(label)

	x_train = np.array(x_train)
	y_train = np.array(y_train)


	class ChatDataset(Dataset):
		def __init__(self):
			self.n_samples = len(x_train)
			self.x_data = x_train
			self.y_data = y_train

		def __getitem__(self, index):
			return self.x_data[index], self.y_data[index]

		def __len__(self):
			return self.n_samples


	batch_size = 8
	hidden_size = 8
	output_size = len(tags)
	input_size = len(x_train[0])
	learning_rate = 0.001
	num_epochs = 1000

	dataset = ChatDataset()
	train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

	model = NeuralNet(input_size, hidden_size, output_size).to(device)

	# Loss and optimizer
	criterion = nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

	# Train the model
	for epoch in range(num_epochs):
		for (words, labels) in train_loader:
			words = words.to(device)
			labels = labels.to(dtype=torch.long).to(device)
			
			outputs = model(words)

			loss = criterion(outputs, labels)
			
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			
		if (epoch+1) % 100 == 0:
			print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


	print(f'final loss: {loss.item():.4f}')

	data = {
	"model_state": model.state_dict(),
	"input_size": input_size,
	"hidden_size": hidden_size,
	"output_size": output_size,
	"all_words": all_words,
	"tags": tags
	}

	#os.remove("data.pth")
	FILE = f"data.pth"

	torch.save(data, FILE)

	print(f'Training is complete, here is the model {FILE}')
 
train_model()

Epoch [100/1000], Loss: 0.3855
Epoch [200/1000], Loss: 0.0100
Epoch [300/1000], Loss: 0.0030
Epoch [400/1000], Loss: 0.0029
Epoch [500/1000], Loss: 0.0008
Epoch [600/1000], Loss: 0.0007
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0004
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0002
final loss: 0.0002
Training is complete, here is the model data.pth


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]


model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

def get_response(msg):

    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                answer = random.choice(intent['responses'])
                return answer
    else:
        return "tushunmadim"
